In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
 
QiskitRuntimeService.save_account(
token="", # Use the 44-character API_KEY you created and saved from the IBM Quantum Platform Home dashboard
instance="crn:v1:bluemix:public:quantum-computing:us-east:a/90ac20c18f6f4077b1d0e84d774560c2:436c3853-3350-418c-9ef5-8bac3f6cda22::", # Optional
)

from qiskit_ibm_runtime import QiskitRuntimeService

# Run every time you need the service
service = QiskitRuntimeService()

# IBM Quantum Computer Workflow Tutorial

This notebook demonstrates the complete 4-step process for running quantum programs on IBM quantum computers using Qiskit patterns:

1. **Map the problem to a quantum-native format**
2. **Optimize the circuits and operators**
3. **Execute using a quantum primitive function**
4. **Analyze the results**

We'll start with a simple Bell state example to understand quantum entanglement, then show how this applies to quantum neural networks.

## Setup and Import Libraries

First, we'll import all the necessary Qiskit libraries for working with IBM quantum computers.

In [ ]:
# Core Qiskit imports
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager

# IBM Quantum Runtime imports  
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit_ibm_runtime.fake_provider import FakeFez

# Visualization and analysis
import matplotlib.pyplot as plt
import numpy as np

print("All libraries imported successfully!")
print("Qiskit version:", __import__("qiskit").__version__)

## Step 1: Map Problem to Quantum-Native Format

### Create a Bell State Circuit

A Bell state is a maximally entangled two-qubit quantum state. We'll create a circuit that produces the Bell state |Φ⁺⟩ = (|00⟩ + |11⟩)/√2.

**Note on bit ordering:** Qiskit uses LSb 0 bit numbering where the nth digit has value 2^n.

In [ ]:
# Create a new circuit with two qubits
qc = QuantumCircuit(2)

# Add a Hadamard gate to qubit 0 (creates superposition)
qc.h(0)

# Perform a controlled-X gate on qubit 1, controlled by qubit 0 (creates entanglement)
qc.cx(0, 1)

# Display the circuit
print("Bell State Circuit:")
print(qc.draw())

# Also create a matplotlib visualization
qc.draw("mpl")

### Define Observable Operators

To measure quantum entanglement, we'll create Pauli operators. These operators allow us to measure expectation values and correlations between qubits.

**Operator Notation:**
- `ZZ` = Z⊗Z: measures correlation between Z on both qubits
- `IZ` = I⊗Z: measures Z on qubit 0 only  
- `ZI` = Z⊗I: measures Z on qubit 1 only

For a perfect Bell state:
- ⟨ZZ⟩ should be +1 (perfect correlation)
- ⟨IZ⟩ and ⟨ZI⟩ should be 0 (no individual bias)

In [ ]:
# Set up six different observables to measure quantum correlations
observables_labels = ["IZ", "IX", "ZI", "XI", "ZZ", "XX"]
observables = [SparsePauliOp(label) for label in observables_labels]

print("Created observables:")
for i, (label, obs) in enumerate(zip(observables_labels, observables)):
    print(f"{i+1}. {label}: {obs}")
    
print(f"\nTotal observables: {len(observables)}")

## Step 2: Optimize Circuits and Operators

### Configure IBM Quantum Backend

Before running on real hardware, we need to:
1. Connect to IBM Quantum service
2. Select an appropriate backend
3. Optimize our circuit for the hardware constraints

**Note:** You need to have saved your IBM Quantum credentials first. If you haven't done this yet, run:
```python
QiskitRuntimeService.save_account(channel="ibm_quantum", token="YOUR_TOKEN_HERE")
```

In [ ]:
# Connect to IBM Quantum service
try:
    service = QiskitRuntimeService()
    
    # Get available backends
    backends = service.backends(simulator=False, operational=True)
    print("Available real quantum backends:")
    for backend in backends[:5]:  # Show first 5
        status = backend.status()
        print(f"  - {backend.name}: {status.pending_jobs} jobs queued, "
              f"{backend.configuration().n_qubits} qubits")
    
    # Select the least busy backend
    backend = service.least_busy(simulator=False, operational=True)
    print(f"\nSelected backend: {backend.name}")
    print(f"Number of qubits: {backend.configuration().n_qubits}")
    print(f"Queue length: {backend.status().pending_jobs}")
    
    real_backend_available = True
    
except Exception as e:
    print(f"Could not connect to IBM Quantum service: {e}")
    print("We'll use a fake backend for demonstration...")
    backend = FakeFez()  # 27-qubit fake backend
    real_backend_available = False
    
print(f"\nUsing backend: {backend.name}")
print(f"Coupling map: {backend.configuration().coupling_map}")
print(f"Basis gates: {backend.configuration().basis_gates}")

### Optimize Circuit for Hardware

Now we'll transpile our circuit to match the backend's instruction set architecture (ISA). This process:
- Maps our logical qubits to physical qubits
- Converts gates to the backend's basis gates
- Optimizes circuit depth and gate count
- Handles qubit connectivity constraints

In [ ]:
# Convert to an ISA circuit and layout-mapped observables
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

print("Original circuit:")
print(f"  Depth: {qc.depth()}")
print(f"  Gate count: {qc.count_ops()}")

# Transpile the circuit
isa_circuit = pm.run(qc)

print(f"\nOptimized ISA circuit:")
print(f"  Depth: {isa_circuit.depth()}")
print(f"  Gate count: {isa_circuit.count_ops()}")
print(f"  Layout: {isa_circuit.layout}")

# Visualize the optimized circuit
print("\nOptimized circuit diagram:")
isa_circuit.draw("mpl", idle_wires=False)

## Step 3: Execute Using Quantum Primitives

### Execute Circuit with Estimator

The Estimator primitive allows us to measure expectation values of observables. We'll configure it with:
- Resilience level 1 (basic error mitigation)
- 5000 shots for good statistics
- Proper observable mapping to match the circuit layout

In [ ]:
# Construct the Estimator instance
if real_backend_available:
    estimator = Estimator(mode=backend)
    print("Using real quantum hardware!")
else:
    estimator = Estimator(backend)
    print("Using fake backend for demonstration")

# Configure Estimator options for better results
estimator.options.resilience_level = 1  # Basic error mitigation
estimator.options.default_shots = 5000  # Number of measurement shots

print(f"Estimator configured:")
print(f"  Resilience level: {estimator.options.resilience_level}")
print(f"  Default shots: {estimator.options.default_shots}")

# Map observables to the circuit layout
mapped_observables = [
    observable.apply_layout(isa_circuit.layout) for observable in observables
]

print(f"Mapped {len(mapped_observables)} observables to circuit layout")

In [ ]:
# Submit the job to the quantum computer
print("Submitting job to quantum backend...")
print("⚠️  Note: Real hardware jobs may take time due to queue wait!")

# Create a PUB (Primitive Unified Bloc) with circuit and observables
job = estimator.run([(isa_circuit, mapped_observables)])

# Print job information
print(f"Job submitted successfully!")
print(f"Job ID: {job.job_id()}")

if real_backend_available:
    print(f"Backend: {backend.name}")
    print(f"Queue position: {backend.status().pending_jobs}")
    print("Waiting for job completion...")
else:
    print("Running on simulator - should complete quickly...")

# Wait for job completion and get results
job_result = job.result()
pub_result = job_result[0]

print("Job completed! ✅")

### Alternative: Simulator Execution

For testing and development, you can run the same circuit on a local simulator. This is faster and doesn't require queue waiting, but won't show real quantum noise effects.

In [ ]:
# Alternative: Run on simulator for comparison
print("Running the same circuit on a simulator for comparison...")

# Use FakeFez backend (simulates a real device with noise)
sim_backend = FakeFez()
sim_estimator = Estimator(sim_backend)

# Optimize circuit for simulator
sim_pm = generate_preset_pass_manager(backend=sim_backend, optimization_level=1)
sim_isa_circuit = sim_pm.run(qc)

# Map observables for simulator
sim_mapped_observables = [
    observable.apply_layout(sim_isa_circuit.layout) for observable in observables
]

# Run simulation
sim_job = sim_estimator.run([(sim_isa_circuit, sim_mapped_observables)])
sim_result = sim_job.result()
sim_pub_result = sim_result[0]

print("Simulator job completed! ✅")
print(f"Simulator backend: {sim_backend.name}")
print(f"Simulated {sim_backend.configuration().n_qubits} qubits with noise model")

## Step 4: Analyze and Visualize Results

Now we'll extract the expectation values and standard deviations from our quantum measurements and visualize the results to see quantum entanglement in action!

In [ ]:
# Extract results from hardware/real backend
values = pub_result.data.evs  # Expectation values
errors = pub_result.data.stds  # Standard deviations

# Extract results from simulator
sim_values = sim_pub_result.data.evs
sim_errors = sim_pub_result.data.stds

print("QUANTUM ENTANGLEMENT MEASUREMENT RESULTS")
print("=" * 50)
print(f"Observable    Hardware/Real     Simulator     Expected")
print("-" * 50)

# Expected values for perfect Bell state
expected_values = [0, 0, 0, 0, 1, 1]  # IZ, IX, ZI, XI, ZZ, XX

for i, label in enumerate(observables_labels):
    hw_val = values[i] if i < len(values) else 0
    sim_val = sim_values[i] if i < len(sim_values) else 0
    exp_val = expected_values[i]
    print(f"{label:10}    {hw_val:8.3f}±{errors[i]:.3f}    {sim_val:8.3f}±{sim_errors[i]:.3f}    {exp_val:8.1f}")

print("\nInterpretation:")
print("• IZ, IX, ZI, XI should be ≈ 0 (no individual qubit bias)")
print("• ZZ, XX should be ≈ 1 (perfect correlation - sign of entanglement)")
print("• Deviations from expected values indicate quantum noise/errors")

In [ ]:
# Create comprehensive visualization
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Hardware vs Simulator comparison
x_pos = np.arange(len(observables_labels))
width = 0.35

ax1.bar(x_pos - width/2, values, width, yerr=errors, 
        label='Hardware/Real', alpha=0.8, capsize=5)
ax1.bar(x_pos + width/2, sim_values, width, yerr=sim_errors, 
        label='Simulator', alpha=0.8, capsize=5)
ax1.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax1.set_xlabel('Observable')
ax1.set_ylabel('Expectation Value')
ax1.set_title('Hardware vs Simulator Results')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(observables_labels)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Expected vs Measured (Hardware)
ax2.plot(observables_labels, expected_values, 'ro-', label='Expected (Perfect Bell State)', markersize=8)
ax2.errorbar(observables_labels, values, yerr=errors, fmt='bo-', 
             label='Measured (Hardware)', markersize=8, capsize=5)
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax2.set_xlabel('Observable')
ax2.set_ylabel('Expectation Value')
ax2.set_title('Expected vs Measured Values')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Error comparison
ax3.bar(x_pos, errors, alpha=0.7, label='Hardware Errors')
ax3.bar(x_pos, sim_errors, alpha=0.7, label='Simulator Errors')
ax3.set_xlabel('Observable')
ax3.set_ylabel('Standard Deviation')
ax3.set_title('Measurement Uncertainties')
ax3.set_xticks(x_pos)
ax3.set_xticklabels(observables_labels)
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate entanglement fidelity
zz_measured = values[4]  # ZZ correlation
xx_measured = values[5]  # XX correlation
entanglement_fidelity = (zz_measured + xx_measured) / 2
print(f"\n🎯 Entanglement Quality Metrics:")
print(f"Bell State Fidelity: {entanglement_fidelity:.3f} (1.0 = perfect)")
print(f"ZZ Correlation: {zz_measured:.3f} ± {errors[4]:.3f}")
print(f"XX Correlation: {xx_measured:.3f} ± {errors[5]:.3f}")

if entanglement_fidelity > 0.8:
    print("✅ High-quality entanglement achieved!")
elif entanglement_fidelity > 0.5:
    print("⚠️  Moderate entanglement with some noise")
else:
    print("❌ Poor entanglement - high noise or errors")

## Connection to Quantum Neural Networks

This same 4-step workflow applies to quantum neural networks:

### 1. **Quantum-Native Format**
- QNNs use parameterized quantum circuits (PQCs) 
- Feature maps encode classical data into quantum states
- Ansatz circuits (like our conv/pool layers) provide trainable parameters

### 2. **Circuit Optimization**
- Transpilation for hardware connectivity
- Gate optimization and error mitigation
- Parameter reduction for noise resilience

### 3. **Primitive Execution**
- Estimator for expectation values (classification/regression)
- Sampler for probability distributions (sampling tasks)
- Batch execution for training efficiency

### 4. **Results Analysis**
- Gradient computation for optimization
- Barren plateau detection
- Performance metrics and visualization

**Key Insight:** The Bell state entanglement we measured here is similar to the quantum correlations that QNNs exploit for machine learning tasks!

## Summary and Next Steps

🎉 **Congratulations!** You've successfully run a quantum program on IBM quantum hardware using the complete Qiskit workflow:

### ✅ What We Accomplished:
1. **Created** a Bell state quantum circuit
2. **Optimized** it for real quantum hardware  
3. **Executed** using IBM Quantum primitives
4. **Analyzed** quantum entanglement measurements

### 🔍 Key Observations:
- **Perfect Bell State**: ZZ and XX correlations should be ≈ 1
- **No Individual Bias**: IZ, IX, ZI, XI should be ≈ 0  
- **Hardware Noise**: Real devices show deviations from ideal values
- **Error Mitigation**: Resilience levels help improve results

### 🚀 Next Steps:
1. **Scale Up**: Try larger circuits and more qubits
2. **Explore QNNs**: Apply this workflow to quantum neural networks
3. **Error Mitigation**: Experiment with higher resilience levels
4. **Real Applications**: Use in machine learning, optimization, or chemistry

### 📚 Learn More:
- [IBM Quantum Documentation](https://docs.quantum.ibm.com/)
- [Qiskit Machine Learning](https://qiskit.org/ecosystem/machine-learning/)
- [Quantum Neural Networks Tutorial](../qiskitqnn_example_multipleRun_barren.py)